In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 11.9MB 8.3MB/s 
     |████████████████████████████████| 24.3MB 88.0MB/s 
     |████████████████████████████████| 194kB 8.6MB/s 
     |████████████████████████████████| 235kB 15.6MB/s 
     |████████████████████████████████| 2.2MB 19.0MB/s 
     |████████████████████████████████| 51kB 801kB/s 


In [2]:
import torch
from torch_geometric.datasets import TUDataset

In [3]:
dataset = TUDataset(root = 'data/TUDataset', name = 'MUTAG')

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!
